<a href="https://colab.research.google.com/github/sasha-kap/Events-Analytics/blob/master/GDELT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Provide credentials to the runtime

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


###Use BigQuery via magics
The google.cloud.bigquery library includes a magic command which runs a query and either displays the result or saves it to a variable as a DataFrame.

In [0]:
# Save output in a variable `df`

%%bigquery --project bq-project-248900 df
SELECT
  REGEXP_REPLACE(theme, r',.*', " ") AS theme,
  COUNT(*) AS count
FROM
  `gdelt-bq.gdeltv2.gkg`,
  UNNEST(SPLIT(V2Themes,';')) AS theme
WHERE
  DATE>20150302000000
  AND DATE < 20150304000000
  AND V2Persons like '%Netanyahu%'
GROUP BY
  theme
ORDER BY 2 DESC
LIMIT 300

In [12]:
df.head()

,theme,count
0,GENERAL_GOVERNMENT,33677
1,LEADER,33405
2,TAX_FNCACT_MINISTER,31174
3,TAX_FNCACT_PRESIDENT,25981
4,TAX_FNCACT_PRIME_MINISTER,25560
